In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f""" CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_capillary_win_membership 
  TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH
customer_capillary AS (
  SELECT * 
  FROM {catalog_name}.udp_wcm_silver_capillary.customer_capillary_daily
  WHERE calday >= '2023-12-06' OR proc_date >= '2024-01-20'
  QUALIFY ROW_NUMBER() OVER (PARTITION BY {catalog_name}.default.FN_CLEAN_MOBILE_NO(mobile) ORDER BY proc_date DESC) = 1
)
SELECT
  a.*,
  IF(b.mobile IS NOT NULL OR c.mobile_number IS NOT NULL OR d.mobile IS NOT NULL, 0, 1) IS_MEMBERSHIP
FROM
  CUSTOMER_CAPILLARY a
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_capillary_plg_member_only b ON
    {catalog_name}.default.FN_CLEAN_MOBILE_NO(a.mobile) = {catalog_name}.default.FN_CLEAN_MOBILE_NO(b.mobile) AND
    a.calday <= '2023-12-14'
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_capillary_membership_exclude c ON
    {catalog_name}.default.FN_CLEAN_MOBILE_NO(a.mobile) = {catalog_name}.default.FN_CLEAN_MOBILE_NO(c.MOBILE_NUMBER) AND
    a.calday <= '2024-02-01'
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_capillary_membership_delete d ON
    {catalog_name}.default.FN_CLEAN_MOBILE_NO(a.mobile) = {catalog_name}.default.FN_CLEAN_MOBILE_NO(d.mobile)
    """)